# Task 4

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import root_mean_squared_error
from keras._tf_keras.keras.models import load_model
from UsefulFunctions import rolling_forecast_multivariate, Optimizer_NonProsumer, profits_from_SOC_strategy

### Step 1: We load our best LSTM model
This is a single layer lstm model with 3 exogenous variables

In [ ]:
### Load the model and data, and make predictions ###

# Load the model
model = load_model('Best_lstm_model.keras')

# Get caracteristic parameters from the model
model_params = model.get_config()
window_size = model_params['layers'][0]['config']['batch_shape'][1]
n_lookahead = model_params['layers'][2]['config']['units']
n_neurons = model_params['layers'][1]['config']['units']
n_features = model_params['layers'][0]['config']['batch_shape'][2]
dropout = model_params['layers'][1]['config']['dropout']

# Print model parameters
print(f"Window size: {window_size}")
print(f"Number of lookahead steps: {n_lookahead}")
print(f"Number of neurons: {n_neurons}")
print(f"Number of features: {n_features}")
print(f"Dropout: {dropout}")

# Load price data
df_prices = pd.read_csv("Elspotprices2nd.csv")
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices.set_index("HourUTC", inplace=True)
df_prices = df_prices.sort_index()

# Load exogenous data
df_exo = pd.read_csv("ProdConData.csv")
df_exo["HourUTC"] = pd.to_datetime(df_exo["HourUTC"])
df_exo.set_index("HourUTC", inplace=True)
df_exo = df_exo.sort_index()

# Merge datasets
df_combined = df_prices.join(df_exo, how='inner')

# Select target + exogenous features
exogenous_vars = ["GrossConsumptionMWh", "OffshoreWindGe100MW_MWh", "SolarPowerGe40kW_MWh"]
features = ["SpotPriceDKK"] + exogenous_vars
df_combined = df_combined[features].dropna()

# Train/test split
train = df_combined.loc[:"2024-08-31"]
test = df_combined.loc["2024-09-01":"2024-09-30"]

# Normalize
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# Fit a separate scaler for the target variable to inverse transform the forecasted values back to original scale
# This ensures that the predictions we make can be transformed back to the original scale
scaler_y = MinMaxScaler()
scaler_y = scaler_y.fit(train["SpotPriceDKK"].values.reshape(-1, 1))

# Forecast
pred_scaled = rolling_forecast_multivariate(model, train_scaled, 
                          test_scaled, window_size, n_lookahead, n_features)

# Inverse transform only price predictions
pred_inv = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1))[:,0]

# Ground truth
true_values = test["SpotPriceDKK"].values

# RMSE for the forecasted values
RMSE_F = root_mean_squared_error(true_values, pred_inv)  
print(f"LSTM (with exogenous vars) RMSE: {RMSE_F:.2f}")

### Step 2: determine the best SOC strategy using the last month of the training dataset

In [ ]:
### Find the optimal SOC strategy ###
# ie the maximum of the function over the range of allowed SOC: [0.2, 2].

# Get data for the SOC strategy
prices_for_soc = df_prices.loc["2024-08-01":"2024-08-31"]
prices_for_soc = prices_for_soc["SpotPriceDKK"].values

# Use the bounded method to find the maximum profit
# within the bounds [0.2, 2]
fun = lambda SOC_strategy: profits_from_SOC_strategy(prices_for_soc, SOC_strategy, negative=True)
result = minimize_scalar(fun, bounds=(0.2, 2), method='bounded')

optimal_SOC_strategy = result.x
optimal_profit = -result.fun
print(f"Optimal SOC strategy: {optimal_SOC_strategy:.2f}")
print(f"Optimal profit: {optimal_profit:.2f}")

In [ ]:
### Plotting the profit as a function of SOC strategy ###

# Define the range of SOC strategies to evaluate
SOC_strategies = np.linspace(0.2, 2, 50)

# Calculate profits for the range of SOC strategies
profits = [profits_from_SOC_strategy(prices_for_soc, soc, negative=False) for soc in SOC_strategies]

# Create the plot
plt.figure(figsize=(10, 4))
plt.plot(SOC_strategies, profits, label="Profit vs SOC Strategy", color="blue", linewidth=2)

# Highlight the optimal SOC strategy
plt.axvline(optimal_SOC_strategy, color="red", linestyle="--", label=f"Optimal SOC: {optimal_SOC_strategy:.2f}")
plt.scatter(optimal_SOC_strategy, optimal_profit, color="red", label=f"Optimal Profit: {optimal_profit:.0f} DKK")

# Add labels, title, and legend
plt.title("Profit as a Function of EOD SOC Strategy", fontsize=16)
plt.xlabel("SOC Strategy (MWh)", fontsize=14)
plt.ylabel("Profit (DKK)", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle="--", alpha=0.6)

# Show the plot
plt.show()

### Step 3: Optimize battery operation on forcasted prices and calculate profits on actual prices

In [ ]:
### Load the data and plot the predicitons ###

# Load data
file_P = os.path.join(os.getcwd(),'Elspotprices2nd.csv')
df_prices = pd.read_csv(file_P)
df_prices["HourUTC"] = pd.to_datetime(df_prices["HourUTC"])
df_prices.rename(columns={'SpotPriceDKK': 'Price'}, inplace=True)

temp_df = df_prices.set_index("HourUTC")
temp_df = temp_df.sort_index()

Persistence = temp_df.loc["2024-08-31":"2024-09-29"].values[:,0]

df_prices = df_prices[
    (df_prices["HourUTC"].dt.year == 2024) & 
    (df_prices["HourUTC"].dt.month == 9)
]

# Plot the predictions, true values, and persistence over the test period
test_data = df_prices['Price'].values

plt.figure(figsize=(10, 4), dpi=100)
plt.plot(np.arange(1, len(pred_inv) + 1), pred_inv, color="blue", label="Forecasted values")
#plt.plot(np.arange(1, len(Persistence) + 1), Persistence, color="green", label="Persistence")
plt.plot(np.arange(1, len(test_data) + 1), test_data, color="red", label="Actual values")
plt.legend(loc="upper right")
plt.grid(alpha=0.25)
plt.xlim(1, len(test_data))
plt.xlabel("Time (hours)")
plt.ylabel("Spot Price (DKK)")
plt.title("Multivariate LSTM Forecast vs Actual Values over the test period")
plt.tight_layout()
plt.show()

In [ ]:
### Calculate profits with the actual, forecasted, and persistence prices ###

# Battery parameters
battery_params = {
    'Pmax': 1,      # Power capacity in MW
    'Cmax': 2,     # Energy capacity in MWh
    'Cmin': 0.2,      # Minimum SOC (10%)
    'C_0': optimal_SOC_strategy,       # Initial SOC
    'C_n': optimal_SOC_strategy,       # Final SOC
    'n_c': 0.95,    # Charging efficiency
    'n_d': 0.95     # Discharging efficiency
}

# Initialize result dictionarie
profits = {'Actual': 0, 'Forcast': 0, 'Actual-forcasted': 0, 'Persistence': 0}

# Get the unique days in the data
days = pd.to_datetime(df_prices['HourUTC'].dt.date.unique())

# Reshape the forcasted prices array to simulate daily data
daily_prices_pred = pred_inv.reshape(-1, 24)  # 30 days, 24 hours each
Persistence = Persistence.reshape(-1, 24)  # 30 days, 24 hours each

# Calculate profits 
for i, day in enumerate(days):
    # Start with 50% SOC for the first day
    if i == 0:
        battery_params['C_0'] = 1
    else:
        battery_params['C_0'] = optimal_SOC_strategy
    
    # Filter data for the current day
    day_date = day.date()
    prices_day = df_prices[df_prices['HourUTC'].dt.date == day_date]
    
    # Extract data
    prices_actual = prices_day['Price'].values
    

    ### Optimize battery operation with actual prices ###
    profit_actual, p_c_actual, p_d_actual, X_actual = Optimizer_NonProsumer(battery_params, prices_actual)
    
    # Calculate the battery net discharge.
    net_discharge_actual = p_d_actual - p_c_actual
    
    # Calculate cost with battery
    day_profit_actual = 0
    for j in range(len(net_discharge_actual)):
        day_profit_actual += net_discharge_actual[j] * prices_actual[j]
    
    profits['Actual'] += day_profit_actual


    ### Optimize battery operation with forcasted prices from our LSTM model ###
    day_prices_pred = daily_prices_pred[i, :]
    profit_pred, p_c_pred, p_d_pred, X_pred = Optimizer_NonProsumer(battery_params, day_prices_pred)

    # Calculate the battery net discharge.
    net_discharge_pred = p_d_pred - p_c_pred

    # Calculate:
    # - predicted profits from forcasted prices
    # - profits from actual prices with optimisation from forcasted prices
    day_profit_pred = 0
    actual_day_profit_pred = 0
    for j in range(len(net_discharge_pred)):
        day_profit_pred += net_discharge_pred[j] * day_prices_pred[j]
        actual_day_profit_pred += net_discharge_pred[j] * prices_actual[j]

    profits['Forcast'] += day_profit_pred
    profits['Actual-forcasted'] += actual_day_profit_pred


    ### Optimize battery operation with price from the persistance model ###
    day_persistence = Persistence[i, :] 
    profit_persi, p_c_persi, p_d_persi, X_persi = Optimizer_NonProsumer(battery_params, day_persistence)

    # Calculate the battery net discharge.
    net_discharge_persi = p_d_persi - p_c_persi

    # Calculate profits from actual prices with optimisation from persistence prices
    actual_day_profit_persi = 0
    for j in range(len(net_discharge_persi)):
        actual_day_profit_persi += net_discharge_persi[j] * prices_actual[j]

    profits['Persistence'] += actual_day_profit_persi

print("--- 4.1 and 4.2 results ---")
print("Profit and optimisation from Actual prices: ", int(profits['Actual']), "DKK")
print("(Not used in report) Profit and optimisation from Forecasted prices: ", int(profits['Forcast']), "DKK") # This is not used in the report
print("Profit from actual prices with optimisation from Forecasted prices: ", int(profits['Actual-forcasted']), "DKK")
print("Profit from actual prices with optimisation from Prsistence prices: ", int(profits['Persistence']), "DKK")